In [28]:
import pandas as pd

# Load the Excel file
file_path = r"D:\data\TYBMS2.xlsx"  # Replace with your Harbour Line file path
df = pd.read_excel(file_path)

# Debug: Display the first few rows of the input data
print("Initial data preview:")
print(df.head())

# Ensure Pincode column is treated as integers
if df['PINCODE'].dtype != 'int64':  # Convert to integers if needed
    df['PINCODE'] = pd.to_numeric(df['PINCODE'], errors='coerce', downcast='integer')

# Drop duplicates in the dataset
df = df.drop_duplicates()
print(f"Dataset rows after dropping duplicates: {df.shape[0]}")

# Specify the custom pincode order
harbour_line_pincodes = [
    400089, 400071, 400088, 400043, 400037, 400703, 400033, 400705, 
    400706, 400015, 400010, 400031, 400614
]

# Define the station-pincode mapping for Harbour Line
harbour_station_mapping = [
    {"STATION": "Chembur, Tilak Nagar", "PINCODE": 400089},
    {"STATION": "Chembur", "PINCODE": 400071},
    {"STATION": "Govandi East, Mankhurd", "PINCODE": 400088},
    {"STATION": "Govandi West", "PINCODE": 400043},
    {"STATION": "Vashi, Turbhe", "PINCODE": 400703},
    {"STATION": "Cotton Green, Reay Road", "PINCODE": 400033},
    {"STATION": "Sanpada, Juinagar, Turbhe", "PINCODE": 400705},
    {"STATION": "Nerul, Sewood Darave", "PINCODE": 400706},
    {"STATION": "Sewri", "PINCODE": 400015},
    {"STATION": "Mazgaon, Dockyard Road", "PINCODE": 400010},
    {"STATION": "Wadala West, Sewri", "PINCODE": 400031},
    {"STATION": "Wadala East, Sion Koliwada ,GTB Nagar", "PINCODE": 400037},
    {"STATION": "CBD Belapur", "PINCODE": 400614},
    
]

# Create a DataFrame from the mapping
mapping_df = pd.DataFrame(harbour_station_mapping)

# Filter the data to include only the specified pincodes
filtered_df = df[df['PINCODE'].isin(harbour_line_pincodes)]

# Debug: Display the filtered data
print("Filtered data preview:")
print(filtered_df.head())
print(f"Rows after filtering by pincodes: {filtered_df.shape[0]}")

# Merge the mapping with the filtered data
merged_df = pd.merge(filtered_df, mapping_df, on="PINCODE", how="left")

# Debug: Check for unmapped rows
unmapped_rows = merged_df[merged_df['STATION'].isnull()]
if not unmapped_rows.empty:
    print("Unmapped rows detected:")
    print(unmapped_rows)

# Group by Station and Pincode to calculate counts
result_df = merged_df.groupby(["STATION", "PINCODE"]).agg(
    COUNT=('PINCODE', 'size'),
    DETAILS=('STATION', 'first')  # To preserve the station name if needed
).reset_index()

# Enforce the custom pincode order
custom_order = {pincode: idx for idx, pincode in enumerate(harbour_line_pincodes)}
result_df["ORDER"] = result_df["PINCODE"].map(custom_order)
result_df = result_df.sort_values(by="ORDER").drop(columns=["ORDER"])

# Save the resulting DataFrame to an Excel file
output_file = r"D:\data\harbour_station_pincode_counts.xlsx"
result_df.to_excel(output_file, index=False, engine="openpyxl")

# Validate counts match
original_row_count = df.shape[0]
filtered_row_count = filtered_df.shape[0]
output_row_count = result_df["COUNT"].sum()

print(f"Original sheet rows: {original_row_count}")
print(f"Filtered sheet rows: {filtered_row_count}")
print(f"Processed sheet rows: {output_row_count}")

if original_row_count == filtered_row_count:
    print("Filtered rows match the original dataset.")
else:
    print("Mismatch between original and filtered rows. Please review the data or the filtering logic.")

if filtered_row_count == output_row_count:
    print("Counts match! Data processed correctly.")
else:
    print("Counts do not match. Please review the data or the mapping.")

print(f"Filtered and arranged Harbour Line data saved to: {output_file}")


Initial data preview:
                                      INSTITUTE NAME  \
0  Bhavans College OR Bhartiya Vidya Bhavans M. M...   
1  Bhavna Trusts Junior and Degree College of Com...   
2  Bhavna Trusts Junior and Degree College of Com...   
3  Dhirajlal Talakchand Sankalchand Shah College ...   
4  GURU NANAK KHALSA COLLEGE OF ARTS,SCIENCE AND ...   

                   STUDENT NAME  GENDER  \
0     DEVADIGA AISHWARYA GANESH  Female   
1  MAPARI IRAM FATIMA TAYYABALI  Female   
2              SAYED UBED RAFIQ    Male   
3        ACHARYA NIREKSHA MOHAN  Female   
4               MARU JAY RAJESH    Male   

                                             ADDRESS  PINCODE      MOBILE  \
0  L/14/108 ,  VRINDAVAN COLONY TILAK NAGAR CHEMB...   400089  8591441018   
1  MG 32/3/8 AMIR BAUG NO 2 PY THORAT MARG RATNAGIRI   400089  9867541453   
2  P L Lokhande Marg Chembur Room No 2 Sayed Burh...   400089  8356972482   
3  P.L LOKHANDE MARG  BEHIND JAI MAHARASHTRA BEKR...   400089  8928696440 